# Prueba Técnica - DonDoctor
#### Web Scraping Extracción Directorio Médico
##### Félix Eduardo Barahona Romero
##### url git: https://github.com/fedbaro/prueba_dondoctor.git
---

#### Importe librerías

In [1]:
# Imports

import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

#### Solicitud ciudad de busqueda

In [2]:
#lista de ciudades disponibles para scrapear en la url https://medicosdoc.com/directorio-salud
lista_ciudades = ['apartado', 'armenia', 'barrancabermeja', 'barranquilla', 'bello', 'bogota', 'bucaramanga', 
                'buenaventura', 'buga', 'cali', 'cartagena', 'cartago', 'caucasia', 'chia', 'choco', 
                'cienega', 'cota', 'cucuta', 'dosquebradas', 'duitama', 'envigado', 'facatativa', 'florencia', 
                'floridablanca', 'fusagasuga', 'girardot', 'giron', 'ibague', 'ipiales', 'itagui', 'jamundi', 
                'la-dorada', 'lorica', 'magangue', 'maicao', 'malambo', 'manizales', 'medellin', 'monteria', 
                'neiva', 'ocaña', 'palmira', 'pasto', 'pereira', 'pitalito', 'popayan', 'quibdo', 'riohacha', 
                'rionegro', 'san-andres', 'santa-marta', 'saravena', 'sincelejo', 'soacha', 'sogamoso', 'soledad', 
                'tulua', 'tunja', 'turbo', 'uribia', 'valledupar', 'villavicencio', 'yopal', 'zipaquira']

#Print lista de ciudades
for c in lista_ciudades:
    print(c)

#Se solicita al usuario ciudad de busqueda
ciudad_busqueda = input('\n \n Ingrese la ciudad de búsqueda como aparece en la lista: ')

apartado
armenia
barrancabermeja
barranquilla
bello
bogota
bucaramanga
buenaventura
buga
cali
cartagena
cartago
caucasia
chia
choco
cienega
cota
cucuta
dosquebradas
duitama
envigado
facatativa
florencia
floridablanca
fusagasuga
girardot
giron
ibague
ipiales
itagui
jamundi
la-dorada
lorica
magangue
maicao
malambo
manizales
medellin
monteria
neiva
ocaña
palmira
pasto
pereira
pitalito
popayan
quibdo
riohacha
rionegro
san-andres
santa-marta
saravena
sincelejo
soacha
sogamoso
soledad
tulua
tunja
turbo
uribia
valledupar
villavicencio
yopal
zipaquira



 
 Ingrese la ciudad de búsqueda como aparece en la lista:  pasto


#### Creación url principal y request BS4

In [3]:
# Creación de url principal de pagina web a scrapear
url = 'https://medicosdoc.com/directorio-salud-'+ciudad_busqueda
url

'https://medicosdoc.com/directorio-salud-pasto'

In [4]:
# Definimos un diccionario para setear los headers
head = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.40'}

# Hacemos petición request cambiando los headers
resp_dm = requests.get(url, headers = head)

In [5]:
# Se valida que los encabezados de la peticón al servidor fueron modificados
resp_dm.request.headers

{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.40', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [6]:
# Validamos estado conexión servidor
print(resp_dm.status_code)

200


In [7]:
#Validamos contenido recibido
#resp_dm.content

In [8]:
#Transformamos contenido html en objeto bs4
soup = BeautifulSoup(resp_dm.content, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [9]:
#validamos soup
#print(soup.prettify())

In [10]:
#Inspeccionamos especialidades y urls
s = BeautifulSoup(resp_dm.text)
subespecialidad = s.find("h5", attrs={'class':'tituloSubespecialidad'})
#print(subespecialidad)

#### Llamado funciones request y tratamiento de datos

In [11]:
# Empaquetamos todo lo anterior en una función y le agregamos código a prueba de fallas
def get_info(url: str) -> (str, str):
  try:
    resp = requests.get(url)
  except Exception as e:
    print("Error en la request.", e)
    return ('', '')
  if resp.status_code != 200:
    print('Error en la respuesta. Status_code = ', resp.status_code)
    return '', ''
  
  s = BeautifulSoup(resp.text)
  try:
    
    #se crea bs tag principal
    #con ciclos for se extrae data de los tag relevantes contenidos en el tag principal
    for heading in s.find_all("ul", attrs={'id':'resultmedicos'}):
        medico = []
        enlace = []
        for nombre in heading.findAll("h3"):            
            medico.append(nombre.text.strip().replace('  ',' '))   
            enlace.append(resp.request.url)
      
    for heading in s.find_all("ul", attrs={'id':'resultmedicos'}):
        direccion = []
        entidad = []
        ciudad = []
        for location in heading.find_all('p', attrs={'class':'doclocation'}):
            direccion.append(location.text.split('\n',2)[1].strip().replace('  ',' '))
            entidad.append(location.text.split('\n',3)[2].strip().replace('  ',' '))
            ciudad.append(location.text.split('\n',4)[3].strip().split(',',1)[0].replace('  ',' '))
            del_ciudad = (location.text.split('\n',4)[3].strip().split(',',1)[0].replace('  ',' '))
            
    for heading in s.find_all("ul", attrs={'id':'resultmedicos'}):
        especialidad = []
        for esp in heading.find_all("h4"):
            especialidad.append(esp.text.strip().replace('  ',' ').replace(' '+del_ciudad,''))
   
    #se retorna una lista con la data requerida
    return medico, especialidad, direccion, entidad, ciudad, enlace
  except:
    print("Error obteniendo datos")
    return '',''   

In [12]:
#Prueba función get_info
#url_prueba = 'https://medicosdoc.com/subespecialidad/cardiologo-tunja'
#get_info(url_prueba)

In [13]:
#se define función para conbvertir a df data de la lista creada por la funcion get_info

def crear_df(lista: list) -> list:
    
    try:
        df_directorio = pd.DataFrame({"medico" : lista[0], "especialidad" : lista[1], 
                              "direccion" : lista[2], "clinica" : lista[3], "ciudad" : lista[4], "url" : lista[5]}, dtype=('string'))    
        return df_directorio
    except:
        print("Error obteniendo datos")
        return None
#crear_df(lista_dir)

In [14]:
#Extracción urls subespecialidades
#se extraen las urls por cada especialidad como se encuentra estructurado en la página
subespecialidades = s.findAll("h5", attrs={'class':'tituloSubespecialidad'})
links = [sub.find('a').get('href') for sub in subespecialidades]
#links

In [15]:
#se valida longitud lista especialidades
len(subespecialidades)

8

In [16]:
#se valida longitud lista links, debe ser igual a la lista subespecialidades
len(links)

8

In [17]:
#se define función principal que realizar requets a cada enlace por especialidad
#se generarn df por cada especialidad
#se concatenan cada uno de los df para crear df final consolidado
def extraer_informacion(links: list) -> list:
    
    data = []
    total = len(links)
    dfs = []
    for i,link in enumerate(links):
        
        if link.startswith('/'):
            link = 'https://medicosdoc.com'+link
        print('[{}/{} - Pidiendo {}'.format(i, total, link))
        
        lista_dir = get_info(link)
        df_dir = crear_df(lista_dir)
        dfs.append(df_dir)

        df_final = pd.concat(dfs)
    
    return df_final

In [18]:
# prueba función final
#enlaces = ['/subespecialidad/cardiologo-tunja', '/subespecialidad/alergologo-tunja', '/subespecialidad/alergologo-pediatra-tunja']
#lista_prueba = extraer_informacion(enlaces)

In [19]:
#lista_prueba

In [20]:
# procesamiento data funcion final, llamado función
df_directorio = extraer_informacion(links)

[0/8 - Pidiendo https://medicosdoc.com/subespecialidad/centros-medicos-pasto
[1/8 - Pidiendo https://medicosdoc.com/subespecialidad/clinica-pasto
[2/8 - Pidiendo https://medicosdoc.com/subespecialidad/cuidados-intensivos-pasto
[3/8 - Pidiendo https://medicosdoc.com/subespecialidad/hospitalizacion-pasto
[4/8 - Pidiendo https://medicosdoc.com/subespecialidad/laboratorio-clinico-pasto
[5/8 - Pidiendo https://medicosdoc.com/subespecialidad/urgencias-pasto
[6/8 - Pidiendo https://medicosdoc.com/subespecialidad/optica-pasto
[7/8 - Pidiendo https://medicosdoc.com/subespecialidad/vacunacion-pasto


#### Limpieza y export dataset

In [21]:
#validación estructura df final
df_directorio.head(5)
#df_directorio

,medico,especialidad,direccion,clinica,ciudad,url
0,"Centro de Habilitación del Niño ""CEHANI""",Ayudas diagnósticas Centros médicos Cirujano C...,Calle 18 No. 45 - 49,,Pasto,https://medicosdoc.com/subespecialidad/centros...
1,Medinuclear,Ayudas diagnósticas Centros médicos Oncólogo,Cra. 34 No. 11 A -12,,Pasto,https://medicosdoc.com/subespecialidad/centros...
0,"Centro de Habilitación del Niño ""CEHANI""",Ayudas diagnósticas Centros médicos Cirujano C...,Calle 18 No. 45 - 49,,Pasto,https://medicosdoc.com/subespecialidad/clinica...
1,Clínica Nuestra Señora de Fatima,Cirujano Clínica Vacunación,Calle 21 No. 26- 40,,Pasto,https://medicosdoc.com/subespecialidad/clinica...
2,Clínica Oncológica Aurora SAS,Clínica Medicina nuclear Oncólogo Radioterapeuta,"Carrera 34 No. 11 A - 12, piso 2-3-5",,Pasto,https://medicosdoc.com/subespecialidad/clinica...


In [22]:
#se eliminan duplicados si existen
df_directorio.drop_duplicates(ignore_index=True, inplace=True)
#df_directorio

In [23]:
#se exporta archivo en formato csv
df_directorio.to_csv('directorio_salud_'+ciudad_busqueda+'.csv', index=False, encoding='utf-8', sep = ';')

In [24]:
#se exporta archivo en formato excel
df_directorio.to_excel('directorio_salud_'+ciudad_busqueda+'.xlsx', index=False, encoding='latin1')